# initialization

In [1]:
import random
import numpy as np
from instance import PDPTWInstance
from solution import PDPTWSolution
from solver import greedy_insertion_init
from operators import RemovalOperators, RepairOperators, ReverseOperators

# 参数设置
n = 10  # pickup点的数量
map_size = 2  # 地图大小
speed = 4  # 车辆速度
extra_time = 10  # delivery点时间窗口起始时间的额外时间
num_vehicles = 5  # 车辆数量
vehicle_capacity = 5  # 车辆容量
battery_capacity = 240  # 电池容量
battery_consume_rate = 1  # 电池消耗率

instance = PDPTWInstance(n, map_size, speed, extra_time, seed=1234)
# instance.plot_instance()
# df = instance.to_dataframe()

In [2]:
initial_solution = greedy_insertion_init(instance, num_vehicles, vehicle_capacity, battery_capacity, battery_consume_rate)
initial_solution.routes

[[0, 7, 17, 6, 16, 8, 4, 14, 18, 9, 19, 0],
 [0, 2, 12, 5, 15, 10, 20, 0],
 [0, 3, 1, 11, 13, 0],
 [0, 0],
 [0, 0]]

In [3]:
initial_solution.objective_function()

683.3159507313269

# Removal

In [4]:
removal_operators = RemovalOperators(initial_solution)
removed_solution, removed_requests = removal_operators.shaw_removal(num_remove=3,p=3)
removed_solution.routes

[[0, 6, 16, 8, 4, 14, 18, 9, 19, 0],
 [0, 2, 12, 10, 20, 0],
 [0, 1, 11, 0],
 [0, 0],
 [0, 0]]

In [5]:
removed_requests

[(3, 13), (7, 17), (5, 15)]

# Repair

In [13]:
repair_operators = RepairOperators(removed_solution)
repair_solution = repair_operators.regret_insertion(removed_requests, k=3)

In [14]:
repair_solution.routes

[[0, 6, 16, 8, 4, 14, 18, 9, 19, 0],
 [0, 2, 12, 10, 20, 0],
 [0, 1, 11, 0],
 [0, 0],
 [0, 0]]

In [11]:
repair_solution.objective_function()

578.1371728214853

In [12]:
repair_solution.is_feasible()

True

# Reverse

In [ ]:
reverse_operators = ReverseOperators(initial_solution)
optimized_solution = reverse_operators.two_opt(max_iterations=100)
optimized_solution.objective_function()

# All

## 1

In [7]:
class RepairOperators:
    def __init__(self, solution):
        self.solution = solution
        self.instance = solution.instance
        self.insertion_log = []  # 用于记录插入日志

    def greedy_insertion(self, removed_pairs):
        for pickup, delivery in removed_pairs:
            best_cost = float('inf')
            best_route = None
            best_insert_position = None

            for vehicle_id, route in enumerate(self.solution.routes):
                for i in range(1, len(route)):
                    for j in range(i, len(route) + 1):
                        temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                        temp_solution = deepcopy(self.solution)
                        temp_solution.routes[vehicle_id] = temp_route
                        temp_solution.update_all()

                        if temp_solution.is_feasible():
                            cost = temp_solution.objective_function()
                            if cost < best_cost:
                                best_cost = cost
                                best_route = vehicle_id
                                best_insert_position = (i, j)

            if best_route is not None and best_insert_position is not None:
                i, j = best_insert_position
                self.solution.routes[best_route] = self.solution.routes[best_route][:i] + [pickup] + self.solution.routes[best_route][i:j] + [delivery] + self.solution.routes[best_route][j:]
                self.solution.update_all()
                self.record_insertion(best_route, pickup, delivery, best_insert_position)  # 记录插入位置

    def regret_insertion(self, removed_pairs, k):
        while removed_pairs:
            max_regret = float('-inf')
            best_request = None
            best_route = None
            best_insert_position = None

            for pickup, delivery in removed_pairs:
                insertion_costs = []
                for vehicle_id, route in enumerate(self.solution.routes):
                    for i in range(1, len(route)):
                        for j in range(i, len(route) + 1):
                            temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                            temp_solution = deepcopy(self.solution)
                            temp_solution.routes[vehicle_id] = temp_route
                            temp_solution.update_all()

                            if temp_solution.is_feasible():
                                cost = temp_solution.objective_function()
                                insertion_costs.append((cost, vehicle_id, i, j))

                # insertion_costs.sort(key=lambda x: x[0])
                # if len(insertion_costs) >= k:
                #     regret = insertion_costs[k-1][0] - insertion_costs[0][0]
                # else:
                #     regret = insertion_costs[-1][0] - insertion_costs[0][0]
                n = len(insertion_costs)
                if n > 0:
                    insertion_costs.sort(key=lambda x: x[0])
                    if n >= k:
                        regret = insertion_costs[k-1][0] - insertion_costs[0][0]
                    else:
                        if n == 1:
                            regret = 0
                        else:
                            regret = insertion_costs[-1][0] - insertion_costs[0][0]
                else:
                    # when insertion_costs is empty
                    regret = float('-inf')

                if regret > max_regret:
                    max_regret = regret
                    best_request = (pickup, delivery)
                    best_route = insertion_costs[0][1]
                    best_insert_position = (insertion_costs[0][2], insertion_costs[0][3])

            if best_request is not None and best_route is not None and best_insert_position is not None:
                removed_pairs.remove(best_request)
                pickup, delivery = best_request
                i, j = best_insert_position
                self.solution.routes[best_route] = self.solution.routes[best_route][:i] + [pickup] + self.solution.routes[best_route][i:j] + [delivery] + self.solution.routes[best_route][j:]
                self.solution.update_all()
                self.record_insertion(best_route, pickup, delivery, best_insert_position)  # 记录插入位置

        
    def record_insertion(self, vehicle_id, pickup, delivery, position):
        """
        记录插入位置
        vehicle_id: 车辆ID
        pickup: 取货点
        delivery: 送货点
        position: 插入位置 (i, j)
        """
        self.insertion_log.append({
        'vehicle_id': vehicle_id,
        'pickup': pickup,
        'delivery': delivery,
        'position': position
        })

    def get_insertion_log(self):
        """
        获取插入日志
        :return: 插入日志
        """
        return self.insertion_log
        

## 2

In [ ]:
class RepairOperators:
    def __init__(self, solution):
        self.solution = deepcopy(solution)
        self.instance = solution.instance
        self.insertion_log = []  # 用于记录插入日志

    #*****************************************************************************************************
    #Start of greedy insertion
    def greedy_insertion(self, removed_pairs):
        # loop the removed pairs
        for pickup, delivery in removed_pairs:
            best_cost = float('inf')
            best_route = None
            best_insert_position = None
            # loop each route to find a suitable location to insert 
            for vehicle_id, route in enumerate(self.solution.routes):
                for i in range(1, len(route)):
                    for j in range(i, len(route) + 1):
                        temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                        temp_solution = deepcopy(self.solution)
                        temp_solution.routes[vehicle_id] = temp_route
                        temp_solution.update_all()

                        if temp_solution.is_feasible():
                            cost = temp_solution.objective_function()
                            if cost < best_cost:
                                best_cost = cost
                                best_route = vehicle_id
                                best_insert_position = (i, j)
            
            # update the self.solution
            if best_route is not None and best_insert_position is not None:
                self.insert_single_request(pickup,delivery,best_route, best_insert_position)
        
        return self.solution
    #*****************************************************************************************************
    #End of greedy insertion

    #*****************************************************************************************************
    #Start of regret insertion
    def regret_insertion(self, removed_pairs, k):

        while removed_pairs:
            max_regret = float('-inf')
            best_request = None
            best_route = None
            best_insert_position = None

            for pickup, delivery in removed_pairs:
                insertion_costs = []
                for vehicle_id, route in enumerate(self.solution.routes):
                    for i in range(1, len(route)):
                        for j in range(i, len(route) + 1):
                            temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                            temp_solution = deepcopy(self.solution)
                            temp_solution.routes[vehicle_id] = temp_route
                            temp_solution.update_all()

                            if temp_solution.is_feasible():
                                cost = temp_solution.objective_function()
                                insertion_costs.append((cost, vehicle_id, i, j))
                
                n = len(insertion_costs)
                if n > 0:
                    insertion_costs.sort(key=lambda x: x[0])
                    if n >= k:
                        regret = insertion_costs[k-1][0] - insertion_costs[0][0]
                    else:
                        if n == 1:
                            regret = 0
                        else:
                            regret = insertion_costs[-1][0] - insertion_costs[0][0]
                else:
                    # when insertion_costs is empty
                    regret = float('-inf')

                if regret > max_regret:
                    max_regret = regret
                    best_request = (pickup, delivery)
                    best_route = insertion_costs[0][1]
                    best_insert_position = (insertion_costs[0][2], insertion_costs[0][3])
                

            if best_request is not None and best_route is not None and best_insert_position is not None:
                removed_pairs.remove(best_request)
                pickup, delivery = best_request
                self.insert_single_request(pickup,delivery,best_route, best_insert_position)
                
        return self.solution
        
    #*****************************************************************************************************
    #End of regret insertion
    def insert_single_request(self, pickup, delivery, vehicle_id, insert_position):
        i, j = insert_position
        self.solution.routes[vehicle_id] = self.solution.routes[vehicle_id][:i] \
                                           + [pickup] + self.solution.routes[vehicle_id][i:j] + [delivery] \
                                           + self.solution.routes[vehicle_id][j:]
        self.solution.update_all() # update all of the things
        self.record_insertion(vehicle_id, pickup, delivery, insert_position)  # 记录插入位置
    
    def record_insertion(self, vehicle_id, pickup, delivery, position):
        """
        记录插入位置
        vehicle_id: 车辆ID
        pickup: 取货点
        delivery: 送货点
        position: 插入位置 (i, j)
        """
        self.insertion_log.append({
        'vehicle_id': vehicle_id,
        'pickup': pickup,
        'delivery': delivery,
        'position': position
        })

    def get_insertion_log(self):
        """
        获取插入日志
        :return: 插入日志
        """
        return self.insertion_log
        

In [8]:
import random
import numpy as np
from copy import deepcopy
from instance import PDPTWInstance
from solution import PDPTWSolution
from solver import greedy_insertion_init
from operators import RemovalOperators

# 参数设置
n = 10  # pickup点的数量
map_size = 2  # 地图大小
speed = 4  # 车辆速度
extra_time = 10  # delivery点时间窗口起始时间的额外时间
num_vehicles = 5  # 车辆数量
vehicle_capacity = 5  # 车辆容量
battery_capacity = 240  # 电池容量
battery_consume_rate = 1  # 电池消耗率

instance = PDPTWInstance(n, map_size, speed, extra_time, seed=1234)
initial_solution = greedy_insertion_init(instance, num_vehicles, vehicle_capacity, battery_capacity, battery_consume_rate)
removal_operators = RemovalOperators(initial_solution)
removed_solution, removed_requests = removal_operators.shaw_removal(num_remove=3,p=3)

In [9]:
removed_requests

[(3, 13), (7, 17), (5, 15)]

In [10]:
repair_operators = RepairOperators(removed_solution)
repair_operators.regret_insertion(removed_requests,k=2)

In [11]:
repair_operators.insertion_log

[{'vehicle_id': 0, 'pickup': 7, 'delivery': 17, 'position': (1, 2)},
 {'vehicle_id': 2, 'pickup': 5, 'delivery': 15, 'position': (2, 2)},
 {'vehicle_id': 2, 'pickup': 3, 'delivery': 13, 'position': (1, 4)}]

In [12]:
removed_solution.routes

[[0, 7, 6, 17, 16, 8, 4, 14, 18, 9, 19, 0],
 [0, 2, 12, 10, 20, 0],
 [0, 3, 1, 5, 15, 13, 11, 0],
 [0, 0],
 [0, 0]]